In [1]:
import sys,os
import numpy as np
import pylab as py
import pandas as pd
from scipy.integrate import quad,fixed_quad
%matplotlib inline
from  matplotlib import rc
from matplotlib.colors import LogNorm
from matplotlib import font_manager
import matplotlib
from matplotlib.pyplot import gca
from matplotlib.ticker import MultipleLocator, FormatStrFormatter,AutoMinorLocator
from scipy.interpolate import interp1d
import random
sizeOfFont = 20
rc('text',usetex=True)
fontProperties = {'weight' : 'normal', 'size' : sizeOfFont}
#ticks_font = matplotlib.font_manager.FontProperties(style='normal',size=sizeOfFont, weight='normal', stretch='normal')
rc('text',usetex=True)
rc('font',**fontProperties)
from scipy.interpolate import interp1d
from iminuit import Minuit
from iminuit.cost import LeastSquares
import warnings
warnings.filterwarnings("ignore")

In [ ]:
dfpythia = pd.read_csv("example.dat")
midpoints = np.array(dfpythia.jT)
errors = np.array(dfpythia.sigma)
values = np.array(dfpythia.exp)

## Construct Theory

In [ ]:
def toy_thy(pT,Nn,pT2):
    return Nn*pT*np.exp(-pT*pT/pT2)

## Perform the fit

In [ ]:
def chi_squared(Nn,pT2):
    res = 0.
    for i in range(len(midpoints)):
        jT = midpoints[i]
        if jT <= 2.:
            theory = toy_thy(jT,Nn,pT2)
            pythia = values[i]
            errval = errors[i]
            res += (theory-pythia)**2./errval**2.0
    return res
chi_squared.errordef = Minuit.LEAST_SQUARES
m = Minuit(chi_squared,Nn = 0.,pT2= 0.)
m.simplex()

In [ ]:
fig = py.figure(figsize=(8,5))
ax = py.subplot(111)
Nnc = 4.44
p2c = 0.44
jT = np.linspace(midpoints[0],midpoints[-1],100)
theory = [toy_thy(jjT,Nnc,p2c) for jjT in jT]
ax.set_xlim(0,2)
ax.set_ylim(0,1.75)
ax.errorbar(midpoints,values,errors,fmt = "r.", label = r"\rm Pythia")
ax.plot(jT,theory, label = r"\rm Theory")
ax.axhline(y = 0, color = "gray")
ax.text(0.75,0.9, r'\rm $e^+ e^-\rightarrow \pi^++$Jet',fontsize = 15,horizontalalignment='center',verticalalignment='center', transform=ax.transAxes)
ax.text(0.75,0.8, r'\rm $N_{event} = 100,000$',fontsize = 15,horizontalalignment='center',verticalalignment='center', transform=ax.transAxes)
ax.text(0.75,0.7, r'\rm $0.2<z_\pi \leq 0.5$',fontsize = 15,horizontalalignment='center',verticalalignment='center', transform=ax.transAxes)
ax.text(0.75,0.6, r'\rm $R = 0.6$',fontsize = 15,horizontalalignment='center',verticalalignment='center', transform=ax.transAxes)
ax.set_ylabel(r'\rm $\sigma^{-1} d\sigma/d P_{\pi \perp}$ (GeV$^{-1}$)',fontsize=sizeOfFont)
ax.set_xlabel(r'\rm $P_{\pi \perp}$ (GeV)',fontsize=sizeOfFont)
ax.legend(frameon = False,fontsize = 20,bbox_to_anchor=(1.4, 0.5))
py.savefig('example.pdf',bbox_inches="tight")